In [ ]:
import agate
import agateexcel
from pathlib import Path
import cchardet as chardet

In [ ]:
def parse_csv(file_path):
    # deactivate type testing, this puts too much constraint on parsing
    # especially for lat/lon columns with commas
    tester = agate.TypeTester(types=(agate.Text, ), limit=0)
    with file_path.open('rb') as f:
        encoding = chardet.detect(f.read()).get('encoding')
    try:
        table = agate.Table.from_csv(file_path, encoding=encoding, sniff_limit=None, column_types=tester)
    except Exception as e:
        print('[x] CSV parse error for %s (%s)' % (file_path, e))
    else:
        return table

In [ ]:
columns_mapping = [
    ('n_amenageur', 'nom_amenageur', 'n_amenageu'), 
    ('n_operateur', 'n_operateu'), 
    ('n_enseigne', ), 
    ('id_station', ), 
    ('n_station', 'nom_station'), 
    ('ad_station', 'adresse_station'), 
    ('code_insee', ), 
    ('Xlongitude', 'longitude_wsg84'), 
    ('Ylatitude', 'latitude_wsg84'), 
    ('nbre_pdc', 'nbre_borne'), 
    ('id_pdc', 'n° borne'), 
    ('puiss_max', ), 
    ('type_prise', 'type_connecteur', 'typ_charge'), 
    ('acces_recharge', 'modalité d\'accès à la borne', 'acces_rech'), 
    ('accessibilité', 'accessibilitã©', 'accessibilite', 'accessibilit�', 'accessibilit‚'), 
    ('observations', ), 
    ('date_maj', ),
]
columns = ['n_amenageur', 'n_operateur', 'n_enseigne', 'id_station', 'n_station', 'ad_station', 'code_insee', 'Xlongitude', 'Ylatitude', 'nbre_pdc', 'id_pdc', 'puiss_max', 'type_prise', 'acces_recharge', 'accessibilité', 'observations', 'date_maj']
columns_low = [x.lower() for x in columns]

In [ ]:
# parse CSV files: analyze difference between standard schema and real data
# infer a mapping of column names variation to match a maximum of data later

# /!\ this is only an analysis step that helped build `columns_mapping` above
# you do not need to run this when doing a consolidation

p = Path('/tmp/out/data')
for child in [x for x in p.iterdir() if x.is_dir()]:
    csvs = list(child.glob('*.csv'))
    for csv in csvs:
        table = parse_csv(csv)
        if table:
            cols = [x.lower() for x in table.column_names]
            missing_pivot = []
            for pivot in ['id_station', 'id_pdc', 'date_maj']:
                if pivot not in cols:
                    missing_pivot.append(pivot)
            if missing_pivot:
                print('Skipping %s for missing pivot %s, cols were %s' % (csv, missing_pivot, cols))
                break
            diff = list(set(cols) - set(columns_low))
            if diff:
                print('DIFF for %s: %s' % (csv, diff))
print('Done.')

In [ ]:
# use columns_mapping to build a database of unique stations based on (id_station, id_pdc, max(date_maj))

bornes = []

def find_by_pivot(row, lines):
    pass

def ifind_in_row_by_col(col, row):
    for key in row.keys():
        if col.lower() == key.lower():
            return row[key]

p = Path('/tmp/out/data')

for child in [x for x in p.iterdir() if x.is_dir()]:
    csvs = list(child.glob('*.csv'))
    for csv in csvs:
        table = parse_csv(csv)
        if table:
            table_cols = [x.lower() for x in table.column_names]
            missing_pivot = []
            for pivot in ['id_station', 'id_pdc', 'date_maj']:
                if pivot not in table_cols:
                    missing_pivot.append(pivot)
            if missing_pivot:
                break
            for row in table.rows:
                borne = {}
                for col in columns_mapping:
                    for c in col:
                        if c.lower() in table_cols:
                            # use the standard name for column
                            borne[col[0]] = ifind_in_row_by_col(c, row)
                            break
                upatt = 'https://www.data.gouv.fr/fr/datasets/%s/#resource-%s'
                borne['source'] = upatt % (csv.parents[0].stem, csv.stem)
                bornes.append(borne)

In [ ]:
len(bornes)

In [ ]:
# deduplicate based on date_maj and id_pdc
from dateutil.parser import parse

unique_bornes = []

def parse_date(date):
    date = str(date)
    try:
        return parse(date)
    except ValueError:
        return parse('1970-1-1')

# filter out id_pdc==None
# TODO maybe use (id_station, id_pdc) instead of only id_pdc
ids = set([b['id_pdc'] for b in bornes if (b['id_pdc'] and str(b['id_pdc']).strip())])
for _id in ids:
    bs = [b for b in bornes if b['id_pdc'] == _id]
    if len(bs) > 1:
        max_date = max([parse_date(b['date_maj']) for b in bs])
        unique = [b for b in bs if parse_date(b['date_maj']) == max_date][0]
    else:
        unique = bs[0]
    ## Data hacks
    # replace `date_maj` original value with parsed date
    unique['date_maj'] = parse_date(unique['date_maj']).strftime('%Y/%m/%d')
    # replace `Xlongitude` and `Xlatitude` commas with points
    unique['Xlongitude'] = unique['Xlongitude'].replace(',', '.') if unique['Xlongitude'] else ''
    unique['Ylatitude'] = unique['Ylatitude'].replace(',', '.') if unique['Ylatitude'] else ''
    unique_bornes.append(unique)

In [ ]:
len(bornes), len([b for b in bornes if b['id_pdc']]), len(unique_bornes)

In [ ]:
# enjoy the results!

import csv

# sort by n_amenageur
unique_bornes.sort(key=lambda k: k['n_amenageur'])

with open('/tmp/out/bornes-IRVE.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns + ['source'], delimiter=';')
    writer.writeheader()
    for b in unique_bornes:
        writer.writerow(b)